TRY:
converting LSTM to Bidirectional layer \
changing the loss function\
changing the embedding size?\
use the full data!



In [ ]:
import pandas as pd 
SOURCE_TWITTER_PATH = "./full_train_source_only.json"
FULL_TWITTER_PATH = "./full_train.json"
Y_TRAIN_PATH = "./train.label.txt"
X_TRAIN_PATH = "./train.data.txt"   # train
SOURCE_STORY_ONLY = True

In [ ]:
def full_data(LABEL_PATH, ID_PATH, DATA_PATH, dropna_on_column="text"):
    """A function that joins data and id and their labels, return a dataframe that trims off twitter we don't have data"""
    # Process Labels
    if LABEL_PATH:
        # 1: Rumour
        # 0: NonRumour
        with open(LABEL_PATH, "r") as f:
            y_label = f.read().strip().split("\n") # remove next line
        y_label = pd.DataFrame(y_label, columns = ["label"])
        y_label[y_label["label"]=="rumour"] = 1
        y_label[y_label["label"]=="nonrumour"] = 0
    
    
    ## Get Dataframe Id
    total_id_list = []
    id_dict       = {}
    with open(ID_PATH, "r") as f:
        for line in f:
            line = line.strip() # remove next line
            line = line.split(',')
            total_id_list.append(line[0])
            id_dict[line[0]] = [line[1:]]
           
    id_df = pd.DataFrame(id_dict).transpose().reindex(total_id_list)
    # join their labels
    if LABEL_PATH:
    # if we are processing testing data, don't add label
        id_df["label"] = y_label["label"]
    
    return id_df

In [ ]:
out = full_data(Y_TRAIN_PATH, X_TRAIN_PATH, FULL_TWITTER_PATH)
out

,0,label
1250219300389974016,"[1250219116993974272, 1250219437027766273, 125...",NaN
554886875303780352,"[554894001946759168, 554959644125167617, 55492...",NaN
1237901309011021825,"[1237901311439450112, 1239862502516760577]",NaN
524958128392376320,"[524961934064754688, 524959028061798401, 52495...",NaN
1239295488677085185,[1239562248990806016],NaN
...,...,...
671181758692507648,"[671200376843067392, 671184263329538048, 67133...",NaN
672513234419638273,"[672513853645717504, 672589162726297600, 67251...",NaN
553508098825261056,"[553511652587360256, 553509546602553344, 55355...",NaN
1249479605582327808,"[1249392473073487873, 1249539822915481602, 124...",NaN


In [ ]:
def join_data_id_label(LABEL_PATH, ID_PATH, DATA_PATH, dropna_on_column="text", SOURCE_STORY_ONLY=False):
    """A function that joins data and id and their labels, return a dataframe that trims off twitter we don't have data"""
    # Process Labels
    if LABEL_PATH:
        # 1: Rumour
        # 0: NonRumour
        with open(LABEL_PATH, "r") as f:
            y_label = f.read().strip().split("\n") # remove next line
        y_label = pd.DataFrame(y_label, columns = ["label"])
        y_label[y_label["label"]=="rumour"] = 1
        y_label[y_label["label"]=="nonrumour"] = 0
    
    
    ## Get Dataframe Id
    total_id_list = []
    with open(ID_PATH, "r") as f:
        for line in f:
            line = line.strip() # remove next line
            if SOURCE_STORY_ONLY:
                line = line.split(',')[0] # split into list\
            else:
                line = line.split(',')
            total_id_list.append(line)
            
    if not SOURCE_STORY_ONLY:
        total_id_list = [item for sublist in total_id_list for item in sublist] # Flat into a single list
    total_id_list = total_id_list
    id_df = pd.DataFrame(total_id_list, columns=["id"])
    id_df["id"] = id_df["id"].astype(str)
    # join their labels
    if LABEL_PATH:
    # if we are processing testing data, don't add label
        id_df["label"] = y_label["label"]
    
    ## Get Twitter Data
    # Read Source Data
    data_df = pd.read_json(DATA_PATH)
    #source_df = pd.concat([source_df, y_train], axis=1)
    data_df["id"] = data_df["id"].astype(str)
    
    ## Join the data file and IDs we got 
    ## Drop rows we don't have data on
    combined_df = id_df.join(data_df.set_index('id'), on='id')
    if dropna_on_column:
        combined_df = combined_df.dropna(subset=['text', 'author_id']).reset_index(drop=True)
    combined_df.reindex(total_id_list) # make sure we have the right order
    return combined_df

In [ ]:
import re
def remove_URL(original):
    """Remove url link in the text"""
    result = re.sub(r"http\S+", "", original)
    result = re.sub(r"www.\S+", "", result)
    result = re.sub(r"wasap.my+", "", result)
    return result

In [ ]:
def preprocess():
    """Function to combine all the preprocessing steps"""
    X_train = join_data_id_label(Y_TRAIN_PATH, X_TRAIN_PATH, FULL_TWITTER_PATH)
    ## 1. Only keep english tweets as most of them are in english
    ## 2. Based on the values count, drop irrelevant features "reply_settings" too
    X_train = X_train[X_train.lang=="en"].reset_index(drop=True)
    # Use only text data and tokenize
    X_train["sentence"] = X_train["text"].apply(remove_URL)
    X_train = X_train[['sentence','label']]
    
    return X_train

In [ ]:
data = preprocess()
data

,sentence,label
0,5. Can regularly rinsing your nose with saline...,0
1,4. Can eating garlic help prevent infection wi...,1
2,6. Do vaccines against pneumonia protect you a...,0
3,7. Can spraying alcohol or chlorine all over y...,0
4,8. How effective are thermal scanners in detec...,0
...,...,...
19835,@rosierawle All good. Just been working out th...,NaN
19836,@rosierawle Best timetable I've heard about! A...,NaN
19837,@lynneSimpkin And this exactly why you need to...,NaN
19838,@rosierawle I might have sent that twice by ac...,NaN


In [ ]:
import random
import numpy as np

sentences = data['sentence'].values
labels = data['label'].values

#partition data into 80/10/10 for train/dev/test
sentences_train, y_train = sentences[:1200], labels[:1200]
sentences_dev, y_dev = sentences[1200:1350], labels[1200:1350]
sentences_test, y_test = sentences[1350:], labels[1350:]

#convert label list into arrays
y_train = np.array(y_train).astype('float32')
y_dev = np.array(y_dev).astype('float32')
y_test = np.array(y_test).astype('float32')



In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(sentences_train)

x_train = tokenizer.texts_to_matrix(sentences_train, mode="count") #BOW representation
vocab_size = x_train.shape[1]

print("Vocab size =", vocab_size)

Vocab size = 5097


In [ ]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.sequence import pad_sequences

maxlen = 50
# Sequence tokenization
xseq_train = tokenizer.texts_to_sequences(sentences_train)
xseq_dev = tokenizer.texts_to_sequences(sentences_dev)
xseq_test = tokenizer.texts_to_sequences(sentences_test)
# Padding
xseq_train = pad_sequences(xseq_train, padding='post', maxlen=maxlen)
xseq_dev = pad_sequences(xseq_dev, padding='post', maxlen=maxlen)
xseq_test = pad_sequences(xseq_test, padding='post', maxlen=maxlen)

In [ ]:
embedding_dim = 10

from keras.layers import LSTM

#word order preserved with this architecture
lstm = Sequential(name="lstm")
lstm.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
lstm.add(LSTM(10))
lstm.add(layers.Dense(1, activation='sigmoid'))
lstm.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
lstm.summary()

Model: "lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 10)            50970     
                                                                 
 lstm (LSTM)                 (None, 10)                840       
                                                                 
 dense (Dense)               (None, 1)                 11        
                                                                 
Total params: 51,821
Trainable params: 51,821
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.fit(xseq_train, y_train, epochs=20, verbose=True, validation_data=(xseq_dev, y_dev), batch_size=10)

loss, accuracy = lstm.evaluate(xseq_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/20
120/120 [==============================] - 2s 16ms/step - loss: 0.0217 - accuracy: 0.9958 - val_loss: 0.5997 - val_accuracy: 0.8733
Epoch 2/20
120/120 [==============================] - 3s 23ms/step - loss: 0.0216 - accuracy: 0.9958 - val_loss: 0.6105 - val_accuracy: 0.8733
Epoch 3/20
120/120 [==============================] - 3s 26ms/step - loss: 0.0183 - accuracy: 0.9967 - val_loss: 0.6607 - val_accuracy: 0.8667
Epoch 4/20
120/120 [==============================] - 3s 25ms/step - loss: 0.0181 - accuracy: 0.9967 - val_loss: 0.6761 - val_accuracy: 0.8667
Epoch 5/20
120/120 [==============================] - 4s 29ms/step - loss: 0.0180 - accuracy: 0.9967 - val_loss: 0.6847 - val_accuracy: 0.8667
Epoch 6/20
120/120 [==============================] - 4s 31ms/step - loss: 0.0179 - accuracy: 0.9967 - val_loss: 0.6979 - val_accuracy: 0.8667
Epoch 7/20
120/120 [==============================] - 3s 27ms/step - loss: 0.0178 - accuracy: 0.9967 - val_loss: 0.6971 - val_accuracy: 0.8667

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(sentences_train)

xtf_train = vectorizer.transform(sentences_train)
xtf_dev = vectorizer.transform(sentences_dev)
xtf_test = vectorizer.transform(sentences_test)

# Padding
xtf_train = pad_sequences(xseq_train, padding='post', maxlen=maxlen)
xtf_dev = pad_sequences(xseq_dev, padding='post', maxlen=maxlen)
xtf_test = pad_sequences(xseq_test, padding='post', maxlen=maxlen)

array([  11,   67,    7,    8,  369,   27,    2,  270, 1094,   57,   24,
        106,    4,   89,    5,   63,  165, 1094,   39,    5,  630,   33,
        175,  254,  447,    9, 1095,    6,    4,   89, 1094,   46,    9,
         39,    5,  175,  208,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
lstm.fit(xtf_train, y_train, epochs=10, verbose=True, validation_data=(xtf_dev, y_dev), batch_size=10)

loss, accuracy = lstm.evaluate(xtf_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
120/120 [==============================] - 2s 15ms/step - loss: 0.0189 - accuracy: 0.9975 - val_loss: 0.4829 - val_accuracy: 0.9067
Epoch 2/10
120/120 [==============================] - 2s 15ms/step - loss: 0.0150 - accuracy: 0.9983 - val_loss: 0.5020 - val_accuracy: 0.9067
Epoch 3/10
120/120 [==============================] - 2s 15ms/step - loss: 0.0095 - accuracy: 0.9992 - val_loss: 0.5238 - val_accuracy: 0.9067
Epoch 4/10
120/120 [==============================] - 2s 15ms/step - loss: 0.0088 - accuracy: 0.9992 - val_loss: 0.4998 - val_accuracy: 0.9133
Epoch 5/10
120/120 [==============================] - 2s 19ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.5188 - val_accuracy: 0.9133
Epoch 6/10
120/120 [==============================] - 2s 15ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.5350 - val_accuracy: 0.9133
Epoch 7/10
120/120 [==============================] - 2s 15ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.5492 - val_accuracy: 0.9133